# Segmentación de Clientes con KMeans

Este notebook está dirigido a estudiantes de **INACAP** y busca explicar paso a paso cómo aplicar **clustering** utilizando el algoritmo **KMeans**.

## Objetivos de Aprendizaje
- Comprender el preprocesamiento de datos para algoritmos de *machine learning*.
- Aplicar el algoritmo **KMeans** para segmentación de clientes.
- Evaluar los resultados con métricas como **Inercia** y **Silhouette Score**.
- Exportar resultados para su análisis.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import os

## Paso 1: Cargar los datos

In [ ]:
# Definimos las constantes
DATA_FILE_PATH = "customer_segmentation.csv"  # archivo de entrada
SUBMISSION_FILE_PATH = "submission.csv"       # archivo de salida
FEATURES = ["purchase_frequency", "average_purchase", "loyalty_points", "months_active"]

def load_data(path):
    print("Paso 1: Cargando datos...")
    return pd.read_csv(path)

df = load_data(DATA_FILE_PATH)
df.head()

## Paso 2: Preprocesamiento de datos
Se escalan las variables para que todas tengan la misma importancia en el modelo.

In [ ]:
def preprocess_data(df):
    print("Paso 2: Escalando datos...")
    X = df[FEATURES].values
    scaler = StandardScaler()
    return scaler.fit_transform(X)

X_scaled = preprocess_data(df)
X_scaled[:5]

## Paso 3: Entrenamiento con KMeans

In [ ]:
def train_kmeans(X, k=3):
    print(f"Paso 3: Entrenando KMeans con k={k}...")
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(X)
    return kmeans, labels

kmeans, labels = train_kmeans(X_scaled, k=3)
labels[:10]

## Paso 4: Evaluación del modelo
Se utilizan dos métricas:
- **Inercia**: mide la cohesión interna de los clusters.
- **Silhouette Score**: mide qué tan bien separados están los clusters.

In [ ]:
def evaluate_model(X, labels, kmeans):
    print("Paso 4: Evaluando modelo...")
    inertia = kmeans.inertia_
    sil_score = silhouette_score(X, labels) if len(set(labels)) > 1 else None
    return inertia, sil_score

inertia, sil_score = evaluate_model(X_scaled, labels, kmeans)
inertia, sil_score

## Paso 5: Exportar resultados
Se generan los resultados en un archivo `.csv` para compartir o analizar más adelante.

In [ ]:
def export_results(kmeans, labels, inertia, sil_score, path):
    print("Paso 5: Exportando resultados...")
    rows = []
    sizes = np.bincount(labels)
    for i, centroid in enumerate(kmeans.cluster_centers_):
        for j, val in enumerate(centroid):
            rows.append([f"cluster{i}_center_{FEATURES[j]}", round(val, 2)])
        rows.append([f"cluster{i}_size", float(sizes[i])])

    rows.append(["inertia", round(inertia, 2)])
    if sil_score is not None:
        rows.append(["silhouette_score", round(sil_score, 2)])

    df_out = pd.DataFrame(rows, columns=["ID", "value"])
    df_out.to_csv(path, index=False)
    print(f"✅ Archivo '{path}' generado con éxito.")

export_results(kmeans, labels, inertia, sil_score, SUBMISSION_FILE_PATH)